# Proyecto Final

***Equipo 07***

- Aide Jazmín González Cruz
- Elena Villalobos Nolasco
- Carolina Acosta Tovany

#### Instrucciones

El proyecto/examen final consistirá en:

La implementación del algoritmo de filtrado colaborativo utilizando la metodología vista en clase (uso de otra metodología no se calificará).

Todos los algoritmos de aprendizaje de máquina que se utilicen deberán haber sido creados por ustedes. Sólo podrán utilizar Transformers y funciones de apoyo de scikit-learn (para realizar la división de los datos en entrenamiento y prueba, o el procedimiento de validación cruzada, etc.) mas ningún estimator (regresión logística, máquina de vectores de soporte, k medias, etc.). 

Se deberá explicar como se obtuvo la k con la que se generó el resultado final.

Se utilizarán los archivos con el conjunto pequeño de calificaciones y películas ubicado en la siguiente https://www.kaggle.com/rounakbanik/the-movies-dataset:

- **links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

- **ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

Con el fin de mejorar la calificación (opcional, puntos extra), se podrán utilizar los algoritmos desarrollado en las tareas del curso y los datos relevantes (los que hacen match con los datos anteriores) contenidos en los archivos:

- **movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

- **keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

- **credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

La métrica con la que se determinará el desempeño del algoritmo es el NDCG 

(https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG)

Una vez obtenida la matriz de calificaciones, el programa deberá ser capaz de regresar las 5 mejores recomendaciones del o de los usuarios que se consulten.

El proyecto se entregará en un Jupyter notebook. El readme file debe contener las instrucciones para que se ejecute el código. Deben cerciorarse que siguiendo esas instrucciones el programa corre sin errores. 

Se deberá subir a la carpeta proyecto_final/equipo_xx en el repositorio GitHub antes de las 7:00 am del día del examen final (14 de diciembre de 2020).    

In [2]:
# Importación de paqueterías necesarias
import pandas as pd
import numpy as np
import random
from sympy import solve
import sympy
from sympy.tensor.array import derive_by_array
from scipy.optimize import fsolve

In [3]:
np.set_printoptions(precision=3, suppress=True)

In [4]:
# Importación de datos
links_small = pd.read_csv('links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
links_small.shape

(9125, 3)

In [6]:
ratings_small = pd.read_csv('ratings_small.csv')
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
ratings_small.shape

(100004, 4)

In [8]:
# Películas en catálogo que no han calificado los usuarios
df_mov_u = pd.DataFrame(ratings_small['movieId'])
df_mov = pd.DataFrame(links_small['movieId'])

In [9]:
common = df_mov.merge(df_mov_u, on=["movieId"])
common

,movieId
0,1
1,1
2,1
3,1
4,1
...,...
99999,161944
100000,162376
100001,162542
100002,162672


In [10]:
result = df_mov[~df_mov.movieId.isin(common.movieId)]
result.shape

(59, 1)

In [11]:
# Construyendo la matriz Y_ai
y_ia = links_small.set_index('movieId').join(ratings_small.set_index('movieId'))
y_ia = y_ia.reset_index()
y_ia

,movieId,imdbId,tmdbId,userId,rating,timestamp
0,1,114709,862.0,7.0,3.0,8.518667e+08
1,1,114709,862.0,9.0,4.0,9.386292e+08
2,1,114709,862.0,13.0,5.0,1.331380e+09
3,1,114709,862.0,15.0,2.0,9.979383e+08
4,1,114709,862.0,19.0,3.0,8.551901e+08
...,...,...,...,...,...,...
100058,162672,3859980,402672.0,611.0,3.0,1.471524e+09
100059,163056,4262980,315011.0,NaN,NaN,NaN
100060,163949,2531318,391698.0,547.0,5.0,1.476419e+09
100061,164977,27660,137608.0,NaN,NaN,NaN


In [12]:
max(y_ia.rating)

5.0

In [13]:
#y_ia.pivot(index="userId", columns="movieId", values="rating") 
y_ia = pd.DataFrame(y_ia.pivot(index='userId', columns='movieId', values='rating'))
y_ia = pd.DataFrame(y_ia.to_records())
y_ia = y_ia[pd.notnull(y_ia['userId'])]
y_ia['userId'] = y_ia['userId'].astype(int)
y_ia = y_ia.drop(['userId'], axis=1)
y_ia

,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
users, movies = y_ia.shape

La función objetivo:
    
$$J(X) = \frac{1}{2} \displaystyle\sum_{(a,i)\in\mathbb{D}} \left(Y_{ai}-\left [ UV^T \right ]_{ai} \right)^2 + \frac{\lambda}{2} \displaystyle\sum_{a=1}^n \displaystyle\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k V_{ij}^2$$

Tenemos una $k = 1$

In [15]:
# Fijando semilla
random.seed(0)

Se crea V de forma aleatoria

In [16]:
# Creando el vector V que son las películas al azar
V = np.random.randint(1,9,size = (1,movies))
V.shape

(1, 9125)

U se definirá con símbolos

In [17]:
u_symbol_vector=[]
for index in range (1, users+1):
    u_symbol = "u"+str(index)
    #print(u_symbol)
    u = sympy.symbols(u_symbol)
    u_symbol_vector.append(u)


In [18]:
# Producto exterior
UV = np.outer(u_symbol_vector,V)

In [19]:
UV

array([[3*u1, 6*u1, 5*u1, ..., 4*u1, 4*u1, 4*u1],
       [3*u2, 6*u2, 5*u2, ..., 4*u2, 4*u2, 4*u2],
       [3*u3, 6*u3, 5*u3, ..., 4*u3, 4*u3, 4*u3],
       ...,
       [3*u669, 6*u669, 5*u669, ..., 4*u669, 4*u669, 4*u669],
       [3*u670, 6*u670, 5*u670, ..., 4*u670, 4*u670, 4*u670],
       [3*u671, 6*u671, 5*u671, ..., 4*u671, 4*u671, 4*u671]],
      dtype=object)

In [23]:
Yia  = y_ia.to_numpy()

Intentemos con U_1 primero

In [39]:
#np.nansum((Yia[0] - UV[0] )**2)
suma_1 = 0
for i in range(1,users+1):
    if (not np.isnan(Yia[0][i])):
        suma_1 += (Yia[0][i] - UV[0][i]) ** 2 
        
suma_1 /= 2

In [43]:
suma_1

6.25*(1 - 0.8*u1)**2

In [44]:
lam = sympy.symbols("lambda")

In [47]:
z = suma_1 + (lam/2 * u_symbol_vector[0])

In [48]:
z

lambda*u1/2 + 6.25*(1 - 0.8*u1)**2

Se deriva 

In [49]:
## No se si se puede hacer esto.. o mejor en un for loop
gf = derive_by_array(z, u_symbol_vector[0])

In [50]:
gf

lambda/2 + 8.0*u1 - 10.0

Se resvuelve...

In [51]:
lam = 1

In [52]:
U = solve(gf)

In [53]:
U

[{lambda: 20.0 - 16.0*u1}]

Después de que se obtiene U, éste se pone fijo para poder calcular V

In [57]:
v_symbol_vector=[]
for index in range (1, movies+1):
    v_symbol = "v"+str(index)
    #print(u_symbol)
    v = sympy.symbols(v_symbol)
    v_symbol_vector.append(v)


In [ ]:
# Producto exterior
UV = np.outer(U,v_symbol_vector)